# Split Videos
Split cholec80 into smaller subvideos to minimize 

In [1]:
import os
import sys
sys.path.append('../../')
from dotmap import DotMap
from moviepy.video.io.VideoFileClip import VideoFileClip

from utils.io import recursive_scan2df, load_yaml

server = 'local'
server = DotMap(load_yaml('../../config/servers.yml')[server])

database_name = 'cholec80'
database = recursive_scan2df(os.path.join(server.database.location, database_name), '.mp4')

# pre-processing
database = database[database.folder != 'sample_videos']
database

,folder,file
0,videos,video36.mp4
1,videos,video37.mp4
2,videos,video52.mp4
3,videos,video38.mp4
4,videos,video53.mp4
...,...,...
75,videos,video33.mp4
76,videos,video34.mp4
77,videos,video50.mp4
78,videos,video51.mp4


In [2]:
def cut_video(input_video_path: str, output_video_path: str, t1: float, t2: float):
    r"""Cuts video and saves clip.

    Args:
        intput_video_path (str): Path to video
        output_video_path (str): Path to output video
        t1 (float): Beginning of clip in seconds
        t2 (float): End of clip in seconds    
    """
    print(input_video_path)
    with VideoFileClip(input_video_path) as video:
        print(output_video_path)
        new = video.subclip(t1, t2)
        new.write_videofile(output_video_path, codec='libx264')

In [3]:
from utils.io import generate_path

# load, split, and safe each video
paths = [os.path.join(server.database.location, database_name, x.folder, x.file) for _, x in database.iterrows()]

split_database_name = 'cholec80_splits'

clip_duration = 60

for path in paths:
    with VideoFileClip(path) as video:
        duration = video.duration
        n_clips = int(duration/clip_duration)

        video_name = video.filename.split('/')[-1].split('.')[0]
        generate_path(os.path.join(server.database.location, split_database_name, video_name))

        for i in range(n_clips):
            start = i*clip_duration
            end = (i+1)*clip_duration

            subclip = video.subclip(start, end)
            subclip.write_videofile(os.path.join(
                server.database.location,
                split_database_name,
                video_name,
                'split_{}.mp4'.format(i)
            ), codec='libx264')


IndentationError: unexpected indent (<ipython-input-3-e4670ebc6ce9>, line 18)